### Paper Choice and Background Outline

- In my project, I will implement the algorithm developed by Dennis D. Boos, Leonard A. Stefanski and Yujun Wu in their article "Fast FSR Variable Selection with Applications to Clinical Trials".

- Many variable selection procedures have been developed in the literature for linear regression models. This paper proposed an updated version of False Selection Rate (FSR) method to control variable selection without simulation. By adding a number of phony variables to the real set of data and monitoring the proportion of the phony variables falsely selected as a function of the tuning parameter, like α-to-enter of forward selection, FSR is able to estimate the appropriate tuning parameter and control the model false selection rate, selecting informative variables and preventing uninformative ones from being selected. Fast FSR in this paper allows us to estimate the tuning parameter from the summary table of the forward selection variable sequence. Therefore, to achieve the same result, no phony variable generation is required in the Fast FSR.

### Pseudocode

- Step 1: Use forward selection to generate the sequence of variables and the associated p-values.

- Step 2: Monotonize the p-value of the original sequence by carrying the larger p-value forward until a even larger p-value. Denote the monotonized p-value sequence with
$$
\tilde{p_1}\leq\tilde{p_2}\leq\cdots\leq\tilde{p_k}\\
$$

- Step 3: For each variable $x_i$ in the selection sequence, calculate the associated

$$
\hat{\alpha_i} = \frac{\gamma(1+S_i)}{k-S_i}  \\
$$

, where $\gamma$ is the pre-determined average selection rate of uninformative variables in the model. $S_i$ is the model size associated with the variables in the sequence. 

- Step 4: Compare $\tilde{p_i}$ and $\hat{\alpha_i}$. Select the model of size $j$, where $j = max\{i: \tilde{p_i}\leq\hat{\alpha_i}\}$. Also, return the corresponding $\hat{\alpha_i}$.

### Draft of Unit Tests 

- Test the algorithm with data from Hammer, S. M. (1996), A trial comparing nucleoside monotherapy with combination therapy in HIV-infected adults with CD4 counts from 200 to 500 per cubic millimeter. The New England Journal of Medicine, 1081-1089, which is saved in http://www4.stat.ncsu.edu/~boos/var.select/actg.175.trt0.txt, and compare my output with the result by Dennis D. Boos and Leonard A. Stefanski.

### Install "leaps" Package

In [1]:
%load_ext rpy2.ipython
from rpy2.robjects.packages import importr
p1=importr('leaps')
p2=importr('stats')

### FSR Algorithm Code

In [53]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from sklearn import datasets, linear_model
%matplotlib inline
%precision 4
plt.style.use('ggplot')

In [55]:
def fsr_fast(x,y,gam0=.05,digits=4):
    
    m = x.shape[1]
    n = x.shape[0]
    if(m >= n):
        m1 = n-5  
    else:
        m1 = m    
    
    pvm = np.zeros(m1)                      # to create pvm below
    
    out_x = p1.regsubsets(x,y,method="forward")
    
    rss = out_x[9]
    nn = x.shape[0]
    vorder = out_x[7]
    
    q = [(rss[i]-rss[i+1])*(nn-i-2)/rss[i+1] for i in range(len(rss)-1)]
    orig = [1-stats.f.cdf(q[i],1,nn-i-2) for i in range(len(rss)-1)]
    
   
    for i in range(0,m1):
        pvm[i] = max(orig[0:i+1])  # sequential max of pvalues
   
    S = np.arange(1,m1+1)
    alpha = gam0*(1+S)/(m1-S)
   
    
    for i in range(0,m1):
        if orig[i]>orig[i+1]:
            i = i+1
        elif pvm[i]<alpha[i] and pvm[i]<gam0:
            i = i+1
        else:
            break
        i = i-1
        
    svorder = np.array(vorder[0:i])-1
    data_x = x.iloc[:,svorder]
    data_x = sm.add_constant(data_x)
    # Train the model using the training sets
    regr = sm.OLS(y,data_x).fit()
    
    return regr, list(data_x.columns.values),svorder
   

###Test with Real Data

In [24]:
import os   
import pandas as pd
if not os.path.exists('ATCG.txt'):
    ! wget http://www4.stat.ncsu.edu/~boos/var.select/actg.175.trt0.txt -O ATCG.txt
data = pd.read_csv('ATCG.txt',delim_whitespace = True).dropna()
data.head()

,Obs,censor,event,age,wtkg,hemo,homo,drugs,karnof,oprior,...,gender,str2,strat,symptom,cd40,cd420,cd496,r,cd80,cd820
0,1,0,1090,43,66.679,0,1,0,100,0,...,1,1,3,0,504,353,660,1,870,782
1,2,1,794,31,73.030,0,1,0,100,0,...,1,1,3,0,244,225,106,1,708,699
2,3,0,957,41,66.226,0,1,1,100,0,...,1,1,3,0,401,366,453,1,889,720
3,4,1,188,35,78.019,0,1,0,100,0,...,1,1,3,0,221,132,-1,0,221,759
4,5,1,308,40,83.009,0,1,0,100,0,...,1,1,3,1,150,90,20,1,1730,1160


In [25]:
data_y = data.ix[:,'event']
data_x = data.ix[:,np.array(['cd40','cd80','age','wtkg','karnof','hemo','homo','drugs','race','gender','str2','symptom'])]

In [26]:
data_x_con = data.ix[:,np.array(['cd40','cd80','age','wtkg','karnof'])]
data_x_con = data_x_con-data_x_con.mean(0)
data_x_con = data_x_con.join(data.ix[:,np.array(['hemo','homo','drugs','race','gender','str2','symptom'])])

In [27]:
data_x.ix[:,'cd40sq']=np.multiply(data_x_con.ix[:,'cd40'],data_x_con.ix[:,'cd40'])
data_x.ix[:,'cd80sq']=np.multiply(data_x_con.ix[:,'cd80'],data_x_con.ix[:,'cd80'])
data_x.ix[:,'agesq']=np.multiply(data_x_con.ix[:,'age'],data_x_con.ix[:,'age'])
data_x.ix[:,'wtkgsq']=np.multiply(data_x_con.ix[:,'wtkg'],data_x_con.ix[:,'wtkg'])
data_x.ix[:,'karnofsq']=np.multiply(data_x_con.ix[:,'karnof'],data_x_con.ix[:,'karnof'])

In [28]:
col = 0
inter = np.zeros(shape=(data_x.shape[0],66))
inter = pd.DataFrame(inter)
for i in np.arange(12):
    for j in np.arange((i+1),12):
        inter.ix[:,col]=data_x_con.ix[:,i]*data_x_con.ix[:,j]
        col = col + 1

data_x = data_x.join(inter)
data_x.head()

,cd40,cd80,age,wtkg,karnof,hemo,homo,drugs,race,gender,...,56,57,58,59,60,61,62,63,64,65
0,504,870,43,66.679,100,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,244,708,31,73.030,100,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,401,889,41,66.226,100,0,1,1,0,1,...,0,1,1,0,0,0,0,1,0,0
3,221,221,35,78.019,100,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,150,1730,40,83.009,100,0,1,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [56]:
fsr_fast(data_x,data_y,gam0=.05,digits=4)[0].summary()

Reordering variables and trying again:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  event   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     24.92
Date:                Thu, 23 Apr 2015   Prob (F-statistic):           4.53e-11
Time:                        22:19:09   Log-Likelihood:                -3810.5
No. Observations:                 532   AIC:                             7627.
Df Residuals:                     529   BIC:                             7640.
Df Model:                           2                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        622.0209     48.880     12.725      0.000       525.998   718.044
cd40           0.8018      0.121      6.620      0.000         0.564     1.040
cd80          -0.1053      0.029     -3.622      0.000        -0.162    -0.048
==============================================================================
Omnibus:                       56.479   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.967
Skew:                          -0.596   Prob(JB):                     4.68e-10
Kurtosis:                       2.280   Cond. No.                     4.12e+03
==============================================================================

Warnings:
[1] The condition number is large, 4.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Method Comparision : Lasso, Ridge, Forward Selection with BIC

In [ ]:
from sklearn.linear_model import Lasso
alpha = 0.5
lasso = Lasso(alpha=alpha, tol=0.001)
y_coef_lasso = lasso.fit(x, y).coef_
y_coef_lasso[np.round(y_coef_lasso,4) != 0].shape


In [ ]:
from sklearn.linear_model import Ridge
alpha = 0.5
ridge = Ridge(alpha=0.5,tol=0.001)
y_coef_ridge = ridge.fit(x, y).coef_
y_coef_ridge[np.round(y_coef_ridge,3) != 0].shape

# Method Assesment and comparison

- Comparsion among lasso, fast fsr, ridge and forward selectino with BIC based on two criteria: Model Error Ratio 
  and False Selection Rate by the simulated data 
- In this simulation study, I simulated 100 data points with 42 variables. Four models are simulated: H1: all variables are zeros. H2: 6 variables are non-zeros at variables 6–8 and 13–15 with values (9,4,1). H3: 10 variables are non-zeros at variables 5–9 and 12–16 with values (25,16,9,4,1). H4: 14 variables are non-zeros at variables 4–10 and 11–17 with value (49, 36, 25, 16, 9, 4, 1)
- Plot the ME and FSR comparison plot for four models

In [ ]:
def ME (method_coef,x,y,n):
    me=[]
    for i in range(n):
        me.append(np.sum((np.dot(method_coef,x.transpose())-y)**2)/150.0)
        i = i + 1
    return me


In [ ]:
# function to find the False Selection Rate  
def FSRR (target,method_selected,n):
    l=[]
    for i in range(n):
      I = set(target)&set(method_selected)
      l.append((len(method_selected)-len(I))/(1.0+len(method_selected)))
      i = i+1
    return l

#  Parall pragramming

In [ ]:
def pi_multiprocessing1(target,method,n):
    """Split a job of length n into num_procs pieces."""
    import multiprocessing
    m = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(m)
    mapfunc = partial(FSRR, target,method)
    results = pool.map(mapfunc,[n/m]*m)
    pool.close()
    return np.mean(results)

In [ ]:
def pi_multiprocessing2(ME,method_coef,x,y,n):
    """Split a job of length n into num_procs pieces."""
    m = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(m)
    mapfunc = partial(ME,method_coef,x,y)
    results = pool.map(mapfunc,[n/m]*m)
    pool.close()
    return np.mean(results),min(min(results))

In [ ]:
# Simulate the data set under model 1
x = pd.DataFrame(np.random.normal(1, 1, 21*150).reshape(150,21))
y = np.random.normal(1, 1, 150)
quad1 = (x.ix[:,0:20])**2
x = np.concatenate((x,quad1),axis=1)
x = pd.DataFrame(x)

alpha = 0.5
lasso = Lasso(alpha=alpha, tol=0.001)
y_coef_lasso = lasso.fit(x, y).coef_
lasso_index = np.where(y_coef_lasso != 0)[0]+1

print fsr_fast(x,y,0.05,4,1)[0].summary()

target =[]
print "LASSO FSR is",pi_multiprocessing1(target,lasso_index,n)
me_r = pi_multiprocessing2(ME,y_coef_lasso,x,y,n)
print me_r
print "LASSO ME is",me_r[1]/me_r[0]
print "FAST FSR is",fsr_fast(x,y,0.05,4,0)[2]

In [ ]:
# Simulate the data under model 2 6–8 and 13–15
x = pd.DataFrame(np.random.normal(1, 1, 21*150).reshape(150,21))
y = 9*x.ix[:,5]+4*x.ix[:,6]+x.ix[:,7]+9*x.ix[:,12]+4*x.ix[:,13]+x.ix[:,14]
quad2 = (x.ix[:,0:20])**2
x = np.concatenate((x,quad2),axis=1)
x = pd.DataFrame(x)

alpha = 0.5
lasso = Lasso(alpha=alpha, tol=0.001)
y_coef_lasso = lasso.fit(x, y).coef_
lasso_index = np.where(y_coef_lasso != 0)[0]+1

print fsr_fast(x,y,0.05,4,1)[0].summary()

n = int(1e5)
target = [6,7,8,13,14,15]

print "LASSO FSR is",pi_multiprocessing1(target,lasso_index,n) ##parallel programming 100 times then average
print "LASSO ME is",pi_multiprocessing2(ME,y_coef_lasso,x,y,n)
print "FAST FSR is",pi_multiprocessing1(target,fsr_fast(x,y,0.05,4,0)[2],n)

In [ ]:
# Simulate the data under model 3 5–9 and 12–16 
x = pd.DataFrame(np.random.normal(1, 1, 21*150).reshape(150,21))
y = 25*x.ix[:,4]+16*x.ix[:,5]+9*x.ix[:,6]+4*x.ix[:,7]+1*x.ix[:,8]+25*x.ix[:,11]+16*x.ix[:,12]+9*x.ix[:,13]+4*x.ix[:,14]+1*x.ix[:,15]
quad3 = (x.ix[:,0:20])**2
x = np.concatenate((x,quad3),axis=1)
x = pd.DataFrame(x)


alpha = 0.5
lasso = Lasso(alpha=alpha, tol=0.001)
y_coef_lasso = lasso.fit(x, y).coef_
lasso_index = np.where(y_coef_lasso != 0)[0]+1



n = int(1e5)
target = [5,6,7,8,9,12,13,14,15,16]
print "LASSO FSR is",pi_multiprocessing1(target,lasso_index,n) ##parallel programming 100 times then average
print "LASSO ME is",pi_multiprocessing2(ME,y_coef_lasso,x,y,n)
print "FAST FSR is",pi_multiprocessing1(target,fsr_fast(x,y,0.05,4,0)[2],n)

In [ ]:
# Simulate the data under model 4  4–10 and 11–17 
x = pd.DataFrame(np.random.normal(1, 1, 21*150).reshape(150,21))
y = 45*x.ix[:,3]+36*x.ix[:,4]+25*x.ix[:,5]+16*x.ix[:,6]+9*x.ix[:,7]+4*x.ix[:,8]+x.ix[:,9]+45*x.ix[:,10]+36*x.ix[:,11]+25*x.ix[:,12]+16*x.ix[:,13]+9*x.ix[:,14]+4*x.ix[:,15]+x.ix[:,16]
quad4 = (x.ix[:,0:20])**2
x = np.concatenate((x,quad4),axis=1)
x = pd.DataFrame(x) 

alpha = 0.5
lasso = Lasso(alpha=alpha, tol=0.001)
y_coef_lasso = lasso.fit(x, y).coef_
lasso_index = np.where(y_coef_lasso != 0)[0]+1

n = int(1e5)
target = [4,5,6,7,8,9,10,11,12,13,14,15,16]
print "LASSO FSR is",pi_multiprocessing1(target,lasso_index,n) ##parallel programming 100000 times then average
print "LASSO ME is",pi_multiprocessing2(ME,y_coef_lasso,x,y,n)
print "FAST FSR is",pi_multiprocessing1(target,fsr_fast(x,y,0.05,4,0)[2],n)

# Profiling the Functions and Optimizing the code by parall pragramming 

In [ ]:
n = int(1e5)
%timeit pi_multiprocessing1(target,lasso_index,n)
%timeit np.mean(FSRR(target,lasso_index,n))

In [ ]:
! pip install --pre line-profiler &> /dev/null
! pip install psutil &> /dev/null
! pip install memory_profiler &> /dev/null

In [ ]:
%timeit -r1 -n1 fsr_fast(x,y,.05,4,0)
%timeit -r1 -n1 io13(interaction)

In [ ]:
stat = %prun -r -q fsr_fast(x,y,.05,4,0)
stat.sort_stats('time').print_stats(10)

In [ ]:
stat.sort_stats('time').print_stats(r'ipython')

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit fsr_fast(x,y,.05,4,0)

# Draft of Make Files:

- make -f MyMakefile
- target: dependencies
- make -f Makefile-1